# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import cartopy.crs as ccrs
import geoviews as gv
import numpy as np


# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("C:/Users/jenna/python-api-challenge/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,invercargill,-46.4000,168.3500,13.90,77,99,1.34,NZ,1701047353
1,1,mount gambier,-37.8333,140.7667,14.43,64,100,4.35,AU,1701047354
2,2,grytviken,-54.2811,-36.5092,2.81,92,34,5.42,GS,1701047354
3,3,fale old settlement,-9.3852,-171.2468,29.86,75,71,7.27,TK,1701047354
4,4,kapa'a,22.0752,-159.3190,24.13,86,75,7.20,US,1701047354


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
# Configure the map plot

map_humidity = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 0.5,
    color = "City"
)

# # Display the map
map_humidity

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_df = pd.DataFrame(city_data_df)
ideal_df = ideal_df[ideal_df["Max Temp"].between(21, 27)]


# Drop any rows with null values
ideal_df.dropna()

# Display sample data
ideal_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
4,4,kapa'a,22.0752,-159.3190,24.13,86,75,7.20,US,1701047354
9,9,adamstown,-25.0660,-130.1015,21.56,86,100,10.47,PN,1701047355
10,10,garissa,-0.4536,39.6401,25.34,84,96,3.05,KE,1701047355
20,20,saint-pierre,-21.3393,55.4781,22.82,60,0,3.09,RE,1701047357
23,23,nova sintra,14.8667,-24.7167,22.35,66,61,6.55,CV,1701047358
...,...,...,...,...,...,...,...,...,...,...
540,540,bardai,11.6500,7.2333,21.70,78,100,2.39,NG,1701047460
545,545,sao joao da barra,-21.6403,-41.0511,23.30,77,100,4.27,BR,1701047461
546,546,ormara,25.2088,64.6357,23.70,51,0,2.62,PK,1701047461
547,547,port shepstone,-30.7414,30.4550,22.62,94,29,3.80,ZA,1701047461


### Step 3: Create a new DataFrame called `hotel_df`.

In [38]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_df.copy()
hotel_df = hotel_df[["City", "Country", "Lat",
                            "Lng", "Humidity"]]



# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""


# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
4,kapa'a,US,22.0752,-159.3190,86,
9,adamstown,PN,-25.0660,-130.1015,86,
10,garissa,KE,-0.4536,39.6401,84,
20,saint-pierre,RE,-21.3393,55.4781,60,
23,nova sintra,CV,14.8667,-24.7167,66,
...,...,...,...,...,...,...
540,bardai,NG,11.6500,7.2333,78,
545,sao joao da barra,BR,-21.6403,-41.0511,77,
546,ormara,PK,25.2088,64.6357,51,
547,port shepstone,ZA,-30.7414,30.4550,94,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [39]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey":geoapify_key,
    "limit":500
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row['Lng']
    latitude = row['Lat']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make an API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
kapa'a - nearest hotel: Pono Kai Resort
adamstown - nearest hotel: No hotel found
garissa - nearest hotel: Hiddig Hotel
saint-pierre - nearest hotel: Tropic Hotel
nova sintra - nearest hotel: Residência Ka Dencho
west island - nearest hotel: Cocos Village Bungalows
arraial do cabo - nearest hotel: No hotel found
cabo san lucas - nearest hotel: Comfort Rooms
namibe - nearest hotel: Hotel Chik Chik Namibe
hadibu - nearest hotel: No hotel found
puerto ayora - nearest hotel: Hostal La Mirada De Solitario George
kayes - nearest hotel: No hotel found
tolanaro - nearest hotel: Hôtel Mahavokey
sydney - nearest hotel: The Fullerton Hotel
visakhapatnam - nearest hotel: Fairfield by Marriot
san antonio de pale - nearest hotel: No hotel found
jaru - nearest hotel: Hotel avenida
carutapera - nearest hotel: No hotel found
sur - nearest hotel: Sur Hotel
masina - nearest hotel: Hôtel
saint-philippe - nearest hotel: Le Baril
mocorito - nearest hotel: No hotel found
coronel du grat

,City,Country,Lat,Lng,Humidity,Hotel Name
4,kapa'a,US,22.0752,-159.3190,86,Pono Kai Resort
9,adamstown,PN,-25.0660,-130.1015,86,No hotel found
10,garissa,KE,-0.4536,39.6401,84,Hiddig Hotel
20,saint-pierre,RE,-21.3393,55.4781,60,Tropic Hotel
23,nova sintra,CV,14.8667,-24.7167,66,Residência Ka Dencho
...,...,...,...,...,...,...
540,bardai,NG,11.6500,7.2333,78,No hotel found
545,sao joao da barra,BR,-21.6403,-41.0511,77,Pousada Mediterrâneo
546,ormara,PK,25.2088,64.6357,51,No hotel found
547,port shepstone,ZA,-30.7414,30.4550,94,The Spot Backpackers'


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [40]:
# Configure the map plot

map_hotels = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 0.5,
    color = "City",
    hover_cols=["City","Hotel Name","Country"]
)



# Display the map
map_hotels



:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)